In [ ]:
# SETUP AUTOMATICO E DOWNLOAD DEL PROGETTO DA GITHUB
import os

# 1. Clona il repository (inserisci il tuo vero link GitHub)
!git clone https://github.com/TUO_NOME_UTENTE/AI-Studio.git

# 2. Sposta l'ambiente di lavoro dentro la cartella del progetto
%cd AI-Studio

# 3. Installa le librerie leggendo il file requirements.txt
!pip install -r requirements.txt
!apt-get install -y ffmpeg

print("✅ Ambiente clonato e pronto all'uso! Tutto il codice è sincronizzato.")

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from google.colab import files
from google.colab import output
output.enable_custom_widget_manager()

# Metadati base
api_key_input = widgets.Password(description="API Key:", style={'description_width': 'initial'})
materia_input = widgets.Text(description="Materia:")
argomento_input = widgets.Text(description="Argomento:")
professore_input = widgets.Text(description="Professore:")

print("⚙️ COMPILA I METADATI E INSERISCI LA CHIAVE API:")
display(api_key_input, materia_input, argomento_input, professore_input)

# Nuova sezione URL
print("\n🔗 OPZIONE A: Scarica da URL (Portale Universitario)")
video_url_input = widgets.Text(description="Link Video:", placeholder="Incolla l'URL qui...", layout=widgets.Layout(width='80%'))
display(video_url_input)
print("Se usi l'URL, carica qui sotto il file cookies.txt:")
cookie_upload = files.upload()

print("\n📤 OPZIONE B: Carica file MP4 dal PC (Ignora se hai usato l'URL)")
video_upload = files.upload()

# Salvataggio nomi file
video_filename = list(video_upload.keys())[0] if video_upload else None
cookie_filename = list(cookie_upload.keys())[0] if cookie_upload else None

print("\n📚 (OPZIONALE) CARICA GLI APPUNTI PRECEDENTI (.tex):")
notes_upload = files.upload()
history_filename = list(notes_upload.keys())[0] if notes_upload else None

In [ ]:
import sys
import os
import yt_dlp
sys.path.append('./src')

from audio_processing import process_video_audio
from vision_processing import extract_keyframes
from llm_engine import generate_notes
from google.colab import files

# LOGICA DI DOWNLOAD CON YT-DLP
if video_url_input.value != "":
    print("⬇️ Download del video dal portale in corso (tramite URL)...")
    video_filename = "lezione_scaricata.mp4"
    ydl_opts = {
        'outtmpl': video_filename,
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best', # Cerca il formato migliore
    }
    # Se hai caricato i cookie, usali!
    if cookie_filename:
        ydl_opts['cookiefile'] = cookie_filename
        
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([video_url_input.value])
        print("✅ Video scaricato con successo sul server!")
    except Exception as e:
        print(f"❌ Errore nel download: {e}")
        video_filename = None
        
if not video_filename or not api_key_input.value:
    print("❌ Errore: Manca il video o la chiave API.")
else:
    print("🎙️ Fase 1: Estrazione e trascrizione audio (VAD attivo)...")
    transcript = process_video_audio(video_filename)
    
    print("👁️ Fase 2: Estrazione fotogrammi chiave (OpenCV)...")
    keyframes = extract_keyframes(video_filename, output_dir="temp_frames")
    
    print("🧠 Fase 3: Generazione LaTeX in corso (Gemini 1.5 Pro)...")
    latex_output = generate_notes(
        transcript=transcript,
        keyframes=keyframes,
        api_key=api_key_input.value,
        prompt_path="prompts/system_prompt.txt",
        history_path=history_filename if history_filename else ""
    )
    
    # Costruzione del file finale e Download automatico
    nome_file = f"{materia_input.value}_{argomento_input.value}_{professore_input.value}".replace(" ", "_") + ".tex"
    
    with open(nome_file, "w", encoding="utf-8") as f:
        f.write(latex_output)
        
    print(f"✅ Appunti generati con successo! Download di {nome_file} in corso...")
    files.download(nome_file) # FORZA IL DOWNLOAD SUL TUO PC LOCALE

In [ ]:
from interactive_agent import run_editor_agent, run_tutor_agent
import ipywidgets as widgets
from IPython.display import display, HTML

# Uniamo tutta la trascrizione in un'unica stringa per dare il contesto
full_transcript = " ".join([seg['text'] for seg in transcript])

mode_selector = widgets.Dropdown(
    options=['Modalità TUTOR (Domande)', 'Modalità EDITOR (Modifica LaTeX)'],
    description='Azione:',
    style={'description_width': 'initial'}
)
text_input = widgets.Textarea(description="Richiesta:", layout=widgets.Layout(width='80%', height='100px'))
latex_input = widgets.Textarea(description="Codice da modificare (solo per Editor):", layout=widgets.Layout(width='80%', height='100px'))
submit_button = widgets.Button(description="Invia all'IA", button_style='success')
output_area = widgets.Output()

def on_submit(b):
    with output_area:
        output_area.clear_output()
        print("Elaborazione in corso...")
        if "TUTOR" in mode_selector.value:
            risposta = run_tutor_agent(text_input.value, full_transcript, api_key_input.value)
            display(HTML(f"<b>👨‍🏫 Il Prof risponde:</b><br><pre style='white-space: pre-wrap;'>{risposta}</pre>"))
        else:
            nuovo_latex = run_editor_agent(latex_input.value, text_input.value, full_transcript, api_key_input.value)
            display(HTML(f"<b>📝 Nuovo Codice LaTeX:</b><br><pre>{nuovo_latex}</pre>"))

submit_button.on_click(on_submit)
display(mode_selector, text_input, latex_input, submit_button, output_area)